In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

## Importing page with Census Codes

In [2]:
url = 'https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697'

In [3]:
response = requests.get(url)

In [4]:
soup = BeautifulSoup(response.content, "lxml")

In [5]:
table = soup.find("div", class_ = "centerColImg")

In [6]:
table.find('tr')

<tr>
<th scope="col">
				FIPS</th>
<th scope="col">
				Name</th>
<th scope="col">
				State</th>
</tr>

In [7]:
col_names = [x.text.replace("\r\n\t\t\t\t", "") for x in table.find('tr').find_all('th')]
col_names
col_names[1] = "CountyName"

In [8]:
rows = []
for tr in table.find_all('tr')[1:]:
    rows.append([x.text.replace("\r\n\t\t\t\t", "") for x in tr.find_all('td')])

In [9]:
df = pd.DataFrame(rows, columns = col_names)

In [10]:
df.head()

,FIPS,CountyName,State
0,01001,Autauga,AL
1,01003,Baldwin,AL
2,01005,Barbour,AL
3,01007,Bibb,AL
4,01009,Blount,AL


In [11]:
df_to_append = pd.DataFrame(
    np.array([["02105", "Hoonah-Angoon", "AK"],
              ["02195", "Petersburg Census Area", "AK"],
              ["02198", "Prince of Wales-Hyder", "AK"],
              ["02230", "Skagway", "AK"],
              ["02275", "Wrangell City and", "AK"], 
              ["08014", "Broomfield", "CO"],
              ["12086", "Miami-Dade", "FL"],
              ["15005", "Kalawao", "HI"]]),  
    columns=['FIPS', 'CountyName', 'State'])

df_to_append

,FIPS,CountyName,State
0,02105,Hoonah-Angoon,AK
1,02195,Petersburg Census Area,AK
2,02198,Prince of Wales-Hyder,AK
3,02230,Skagway,AK
4,02275,Wrangell City and,AK
5,08014,Broomfield,CO
6,12086,Miami-Dade,FL
7,15005,Kalawao,HI


In [12]:
df = pd.concat([df,df_to_append ], axis = 0).reset_index(drop = True)

In [13]:
df.tail()

,FIPS,CountyName,State
3235,02230,Skagway,AK
3236,02275,Wrangell City and,AK
3237,08014,Broomfield,CO
3238,12086,Miami-Dade,FL
3239,15005,Kalawao,HI


## Filling out with additional columns 

In [14]:
df['CountyName_Full'] = df['CountyName'] + " County"

In [15]:
df.tail()

,FIPS,CountyName,State,CountyName_Full
3235,02230,Skagway,AK,Skagway County
3236,02275,Wrangell City and,AK,Wrangell City and County
3237,08014,Broomfield,CO,Broomfield County
3238,12086,Miami-Dade,FL,Miami-Dade County
3239,15005,Kalawao,HI,Kalawao County


### Adding State Names

In [16]:
stnames = pd.read_csv('../data/raw_data/state_abbr.csv')

In [17]:
stnames.head()

,abbr,StateName
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [18]:
final = df.merge(stnames, left_on = 'State', right_on = "abbr").drop("abbr", axis = 1)

In [19]:
final.columns

Index(['FIPS', 'CountyName', 'State', 'CountyName_Full', 'StateName'], dtype='object')

In [20]:
final.columns = ['fips', 'county', 'state_abbr', 'county_full', 'state']

In [21]:
final.sample(5)

,fips,county,state_abbr,county_full,state
158,05089,Marion,AR,Marion County,Arkansas
2493,47121,Meigs,TN,Meigs County,Tennessee
120,05013,Calhoun,AR,Calhoun County,Arkansas
3008,54017,Doddridge,WV,Doddridge County,West Virginia
1267,26065,Ingham,MI,Ingham County,Michigan


## Adding Columns to map FIPS to Census Names of counties

In [22]:
final['f_state_county_full'] = final['state'] + final['county_full']

In [23]:
final.head()

,fips,county,state_abbr,county_full,state,f_state_county_full
0,01001,Autauga,AL,Autauga County,Alabama,AlabamaAutauga County
1,01003,Baldwin,AL,Baldwin County,Alabama,AlabamaBaldwin County
2,01005,Barbour,AL,Barbour County,Alabama,AlabamaBarbour County
3,01007,Bibb,AL,Bibb County,Alabama,AlabamaBibb County
4,01009,Blount,AL,Blount County,Alabama,AlabamaBlount County


In [24]:
census_names = pd.read_csv('../data/Census-County-names').drop('Unnamed: 0', axis = 1)
census_names.head()

,CTYNAME,STNAME
0,Autauga County,Alabama
1,Baldwin County,Alabama
2,Barbour County,Alabama
3,Bibb County,Alabama
4,Blount County,Alabama


In [25]:
census_names["c_state_county"] = census_names['STNAME'] + census_names['CTYNAME']

In [26]:
census_names.head()

,CTYNAME,STNAME,c_state_county
0,Autauga County,Alabama,AlabamaAutauga County
1,Baldwin County,Alabama,AlabamaBaldwin County
2,Barbour County,Alabama,AlabamaBarbour County
3,Bibb County,Alabama,AlabamaBibb County
4,Blount County,Alabama,AlabamaBlount County


In [27]:
print( "census shape", census_names.shape)
print ("fips shape", final.shape)
print ("number of nulls we should get (ideally): ", final.shape[0] - census_names.shape[0])

census shape (3142, 3)
fips shape (3240, 6)
number of nulls we should get (ideally):  98


In [28]:
comb = census_names.merge(final, left_on = 'c_state_county', right_on = 'f_state_county_full', how = 'outer')
comb.shape

(3410, 9)

In [29]:
comb.head()

,CTYNAME,STNAME,c_state_county,fips,county,state_abbr,county_full,state,f_state_county_full
0,Autauga County,Alabama,AlabamaAutauga County,01001,Autauga,AL,Autauga County,Alabama,AlabamaAutauga County
1,Baldwin County,Alabama,AlabamaBaldwin County,01003,Baldwin,AL,Baldwin County,Alabama,AlabamaBaldwin County
2,Barbour County,Alabama,AlabamaBarbour County,01005,Barbour,AL,Barbour County,Alabama,AlabamaBarbour County
3,Bibb County,Alabama,AlabamaBibb County,01007,Bibb,AL,Bibb County,Alabama,AlabamaBibb County
4,Blount County,Alabama,AlabamaBlount County,01009,Blount,AL,Blount County,Alabama,AlabamaBlount County


In [30]:
#this is what the original list says!
comb[comb['c_state_county'].isnull()]["f_state_county_full"][:10]

3142                AlabamaDe Kalb County
3143               AlabamaSt Clair County
3144          AlaskaAleutians East County
3145          AlaskaAleutians West County
3146               AlaskaAnchorage County
3147                  AlaskaBethel County
3148             AlaskaBristol Bay County
3149                  AlaskaDenali County
3150              AlaskaDillingham County
3151    AlaskaFairbanks North Star County
Name: f_state_county_full, dtype: object

In [31]:
comb[comb['c_state_county'].isnull()]["f_state_county_full"].shape  #there are 268 for which fips have no match in census

(268,)

In [32]:
#this is what census says!  #SO THESE ARE THE VALUES I HAVE TO *ADD* to census
comb[comb['f_state_county_full'].isnull()]["c_state_county"][:10]

24                  AlabamaDeKalb County
57               AlabamaSt. Clair County
67          AlaskaAleutians East Borough
68      AlaskaAleutians West Census Area
69          AlaskaAnchorage Municipality
70              AlaskaBethel Census Area
71             AlaskaBristol Bay Borough
72                  AlaskaDenali Borough
73          AlaskaDillingham Census Area
74    AlaskaFairbanks North Star Borough
Name: c_state_county, dtype: object

In [33]:
comb[comb['f_state_county_full'].isnull()]["c_state_county"].shape #there are 173 in c census that don't have a matching fups

(170,)

In [34]:
df_c_to_fs = []

In [35]:
df_c_to_f_1 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][:11]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][:11])})
df_c_to_fs.append(df_c_to_f_1)
df_c_to_f_1


,c_state_county,f_state_county_full
0,AlabamaDeKalb County,AlabamaDe Kalb County
1,AlabamaSt. Clair County,AlabamaSt Clair County
2,AlaskaAleutians East Borough,AlaskaAleutians East County
3,AlaskaAleutians West Census Area,AlaskaAleutians West County
4,AlaskaAnchorage Municipality,AlaskaAnchorage County
5,AlaskaBethel Census Area,AlaskaBethel County
6,AlaskaBristol Bay Borough,AlaskaBristol Bay County
7,AlaskaDenali Borough,AlaskaDenali County
8,AlaskaDillingham Census Area,AlaskaDillingham County
9,AlaskaFairbanks North Star Borough,AlaskaFairbanks North Star County


In [36]:
df_c_to_f_2 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][12:16]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][11:15])})
df_c_to_fs.append(df_c_to_f_2)
df_c_to_f_2

,c_state_county,f_state_county_full
0,AlaskaJuneau City and Borough,AlaskaJuneau County
1,AlaskaKenai Peninsula Borough,AlaskaKenai Peninsula County
2,AlaskaKetchikan Gateway Borough,AlaskaKetchikan Gateway County
3,AlaskaKodiak Island Borough,AlaskaKodiak Island County


In [37]:
#leaving out -- f_state_county_full 16

df_c_to_f_3 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][17:22]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][15:20])})
df_c_to_fs.append(df_c_to_f_3)
df_c_to_f_3

,c_state_county,f_state_county_full
0,AlaskaLake and Peninsula Borough,AlaskaLake and Peninsula County
1,AlaskaMatanuska-Susitna Borough,AlaskaMatanuska-Susitna County
2,AlaskaNome Census Area,AlaskaNome County
3,AlaskaNorth Slope Borough,AlaskaNorth Slope County
4,AlaskaNorthwest Arctic Borough,AlaskaNorthwest Arctic County


In [38]:
#leaving out -- f_state_county_full 22
#leaving out -- c_state_county_full 25


df_c_to_f_4 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][24:25]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][21:22])})
df_c_to_fs.append(df_c_to_f_4)
df_c_to_f_4

,c_state_county,f_state_county_full
0,AlaskaSitka City and Borough,AlaskaSitka County


In [39]:
df_c_to_f_4a = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][26:28]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][23:25])})
df_c_to_fs.append(df_c_to_f_4a)
df_c_to_f_4a

,c_state_county,f_state_county_full
0,AlaskaSoutheast Fairbanks Census Area,AlaskaSoutheast Fairbanks County
1,AlaskaValdez-Cordova Census Area,AlaskaValdez-Cordova County


In [40]:
df_c_to_f_5 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][29:31]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][27:29])})
df_c_to_fs.append(df_c_to_f_5)
df_c_to_f_5

,c_state_county,f_state_county_full
0,AlaskaYakutat City and Borough,AlaskaYakutat County
1,AlaskaYukon-Koyukuk Census Area,AlaskaYukon-Koyukuk County


In [41]:
df_c_to_f_6 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][31:33]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][34:36])})
df_c_to_fs.append(df_c_to_f_6)
df_c_to_f_6

,c_state_county,f_state_county_full
0,ArkansasSt. Francis County,ArkansasSt Francis County
1,District of ColumbiaDistrict of Columbia,District of ColumbiaWashington County


In [42]:
df_c_to_f_7 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][33:124]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][37:128])})
df_c_to_fs.append(df_c_to_f_7)
df_c_to_f_7[70:]

,c_state_county,f_state_county_full
70,LouisianaVernon Parish,LouisianaVernon County
71,LouisianaWashington Parish,LouisianaWashington County
72,LouisianaWebster Parish,LouisianaWebster County
73,LouisianaWest Baton Rouge Parish,LouisianaWest Baton Rouge County
74,LouisianaWest Carroll Parish,LouisianaWest Carroll County
75,LouisianaWest Feliciana Parish,LouisianaWest Feliciana County
76,LouisianaWinn Parish,LouisianaWinn County
77,MarylandPrince George's County,MarylandPrince Georges County
78,MarylandQueen Anne's County,MarylandQueen Annes County
79,MarylandSt. Mary's County,MarylandSt Marys County


In [43]:
df_c_to_f_8 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][124:132]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][129:137])})
df_c_to_fs.append(df_c_to_f_8)
df_c_to_f_8

,c_state_county,f_state_county_full
0,NevadaCarson City,NevadaCarson City County
1,New MexicoDoña Ana County,New MexicoDona Ana County
2,New YorkSt. Lawrence County,New YorkSt Lawrence County
3,North DakotaLaMoure County,North DakotaLa Moure County
4,South DakotaOglala Lakota County,South DakotaShannon County
5,TennesseeDeKalb County,TennesseeDe Kalb County
6,TexasDeWitt County,TexasDe Witt County
7,VirginiaAlexandria city,VirginiaAlexandria City County


In [45]:
df_c_to_f_9 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][132:134]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][138:140])})
df_c_to_fs.append(df_c_to_f_9)
df_c_to_f_9

,c_state_county,f_state_county_full
0,VirginiaBristol city,VirginiaBristol City County
1,VirginiaBuena Vista city,VirginiaBuena Vista City County


In [46]:
df_c_to_f_10 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][134:136]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][140:142])})
df_c_to_fs.append(df_c_to_f_10)
df_c_to_f_10

,c_state_county,f_state_county_full
0,VirginiaCharlottesville city,VirginiaCharlottesville City County
1,VirginiaChesapeake city,VirginiaChesapeake City County


In [47]:

df_c_to_f_11 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][136:163]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][143:170])})
df_c_to_fs.append(df_c_to_f_11)
df_c_to_f_11

,c_state_county,f_state_county_full
0,VirginiaColonial Heights city,VirginiaColonial Heights Cit County
1,VirginiaCovington city,VirginiaCovington City County
2,VirginiaDanville city,VirginiaDanville City County
3,VirginiaEmporia city,VirginiaEmporia City County
4,VirginiaFairfax city,VirginiaFairfax City County
5,VirginiaFalls Church city,VirginiaFalls Church City County
6,VirginiaFranklin city,VirginiaFranklin City County
7,VirginiaFredericksburg city,VirginiaFredericksburg City County
8,VirginiaGalax city,VirginiaGalax City County
9,VirginiaHampton city,VirginiaHampton City County


In [48]:

df_c_to_f_12 = pd.DataFrame({'c_state_county' : np.array( comb[comb['f_state_county_full'].isnull()]["c_state_county"][163:171]), 
                         'f_state_county_full' : np.array(comb[comb['c_state_county'].isnull()]["f_state_county_full"][171:178])})
df_c_to_fs.append(df_c_to_f_12)
df_c_to_f_12

,c_state_county,f_state_county_full
0,VirginiaStaunton city,VirginiaStaunton City County
1,VirginiaSuffolk city,VirginiaSuffolk City County
2,VirginiaVirginia Beach city,VirginiaVirginia Beach City County
3,VirginiaWaynesboro city,VirginiaWaynesboro City County
4,VirginiaWilliamsburg city,VirginiaWilliamsburg City County
5,VirginiaWinchester city,VirginiaWinchester City County
6,WisconsinSt. Croix County,WisconsinSt Croix County


In [49]:
additional_mapping = pd.DataFrame(
    np.array([["AlaskaHoonah-Angoon Census Area", "AlaskaHoonah-Angoon County"],
              ["AlaskaKusilvak Census Area", "AlaskaWade Hampton County"],
              ["AlaskaPetersburg Borough", "AlaskaPetersburg Census Area County"],
              ["AlaskaSkagway Municipality", "AlaskaSkagway County"],
              ["AlaskaWrangell City and Borough", "AlaskaWrangell City and County"]]),
    columns=['c_state_county', 'f_state_county_full'])

# ["District of ColumbiaDistrict of Columbia", "District of ColumbiaWashington County"]

df_c_to_fs.append(additional_mapping)
additional_mapping

,c_state_county,f_state_county_full
0,AlaskaHoonah-Angoon Census Area,AlaskaHoonah-Angoon County
1,AlaskaKusilvak Census Area,AlaskaWade Hampton County
2,AlaskaPetersburg Borough,AlaskaPetersburg Census Area County
3,AlaskaSkagway Municipality,AlaskaSkagway County
4,AlaskaWrangell City and Borough,AlaskaWrangell City and County


In [50]:
df_c_to_f = df_c_to_f_1.copy()
for matchingdf in df_c_to_fs[1:]:
    df_c_to_f = pd.concat([df_c_to_f,matchingdf], axis = 0)  #.reset_index(drop = True)


In [52]:
df_c_to_f[120:]

,c_state_county,f_state_county_full
2,New YorkSt. Lawrence County,New YorkSt Lawrence County
3,North DakotaLaMoure County,North DakotaLa Moure County
4,South DakotaOglala Lakota County,South DakotaShannon County
5,TennesseeDeKalb County,TennesseeDe Kalb County
6,TexasDeWitt County,TexasDe Witt County
7,VirginiaAlexandria city,VirginiaAlexandria City County
0,VirginiaBristol city,VirginiaBristol City County
1,VirginiaBuena Vista city,VirginiaBuena Vista City County
0,VirginiaCharlottesville city,VirginiaCharlottesville City County
1,VirginiaChesapeake city,VirginiaChesapeake City County


In [53]:
df_c_to_f.head()

,c_state_county,f_state_county_full
0,AlabamaDeKalb County,AlabamaDe Kalb County
1,AlabamaSt. Clair County,AlabamaSt Clair County
2,AlaskaAleutians East Borough,AlaskaAleutians East County
3,AlaskaAleutians West Census Area,AlaskaAleutians West County
4,AlaskaAnchorage Municipality,AlaskaAnchorage County


In [55]:
census_names.head()

,CTYNAME,STNAME,c_state_county
0,Autauga County,Alabama,AlabamaAutauga County
1,Baldwin County,Alabama,AlabamaBaldwin County
2,Barbour County,Alabama,AlabamaBarbour County
3,Bibb County,Alabama,AlabamaBibb County
4,Blount County,Alabama,AlabamaBlount County


In [56]:
mapping = df_c_to_f.merge(census_names, on = "c_state_county")[['f_state_county_full', "CTYNAME"]]
mapping.columns = ['f_state_county_full', "census_CTYNAME"]
mapping.head()


,f_state_county_full,census_CTYNAME
0,AlabamaDe Kalb County,DeKalb County
1,AlabamaSt Clair County,St. Clair County
2,AlaskaAleutians East County,Aleutians East Borough
3,AlaskaAleutians West County,Aleutians West Census Area
4,AlaskaAnchorage County,Anchorage Municipality


In [57]:
comb = comb.merge(mapping, on = 'f_state_county_full', how = 'outer')
comb.shape

(3410, 10)

In [58]:
comb.sample(10)

,CTYNAME,STNAME,c_state_county,fips,county,state_abbr,county_full,state,f_state_county_full,census_CTYNAME
1668,Lewis and Clark County,Montana,MontanaLewis and Clark County,30049,Lewis and Clark,MT,Lewis and Clark County,Montana,MontanaLewis and Clark County,NaN
1240,Franklin County,Maine,MaineFranklin County,23007,Franklin,ME,Franklin County,Maine,MaineFranklin County,NaN
50,Southeast Fairbanks Census Area,Alaska,AlaskaSoutheast Fairbanks Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1777,Saunders County,Nebraska,NebraskaSaunders County,31155,Saunders,NE,Saunders County,Nebraska,NebraskaSaunders County,NaN
254,Boone County,Arkansas,ArkansasBoone County,05009,Boone,AR,Boone County,Arkansas,ArkansasBoone County,NaN
1010,Worth County,Iowa,IowaWorth County,19195,Worth,IA,Worth County,Iowa,IowaWorth County,NaN
2341,Somerset County,Pennsylvania,PennsylvaniaSomerset County,42111,Somerset,PA,Somerset County,Pennsylvania,PennsylvaniaSomerset County,NaN
3175,NaN,NaN,NaN,02275,Wrangell City and,AK,Wrangell City and County,Alaska,AlaskaWrangell City and County,Wrangell City and Borough
1855,Luna County,New Mexico,New MexicoLuna County,35029,Luna,NM,Luna County,New Mexico,New MexicoLuna County,NaN
2586,Brooks County,Texas,TexasBrooks County,48047,Brooks,TX,Brooks County,Texas,TexasBrooks County,NaN


In [59]:
comb["census_CTYNAME"] = comb["census_CTYNAME"].fillna(comb["county_full"])

In [60]:
comb.shape

(3410, 10)

In [61]:
comb.sample(10)

,CTYNAME,STNAME,c_state_county,fips,county,state_abbr,county_full,state,f_state_county_full,census_CTYNAME
2649,Goliad County,Texas,TexasGoliad County,48175,Goliad,TX,Goliad County,Texas,TexasGoliad County,Goliad County
273,Franklin County,Arkansas,ArkansasFranklin County,05047,Franklin,AR,Franklin County,Arkansas,ArkansasFranklin County,Franklin County
2571,Bailey County,Texas,TexasBailey County,48017,Bailey,TX,Bailey County,Texas,TexasBailey County,Bailey County
1323,Isabella County,Michigan,MichiganIsabella County,26073,Isabella,MI,Isabella County,Michigan,MichiganIsabella County,Isabella County
99,LaSalle Parish,Louisiana,LouisianaLaSalle Parish,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2789,Trinity County,Texas,TexasTrinity County,48455,Trinity,TX,Trinity County,Texas,TexasTrinity County,Trinity County
2489,Dickson County,Tennessee,TennesseeDickson County,47043,Dickson,TN,Dickson County,Tennessee,TennesseeDickson County,Dickson County
195,Escambia County,Alabama,AlabamaEscambia County,01053,Escambia,AL,Escambia County,Alabama,AlabamaEscambia County,Escambia County
1486,Jones County,Mississippi,MississippiJones County,28067,Jones,MS,Jones County,Mississippi,MississippiJones County,Jones County
778,Logan County,Illinois,IllinoisLogan County,17107,Logan,IL,Logan County,Illinois,IllinoisLogan County,Logan County


#### What Census ones are left?

In [62]:
census_names.head()

,CTYNAME,STNAME,c_state_county
0,Autauga County,Alabama,AlabamaAutauga County
1,Baldwin County,Alabama,AlabamaBaldwin County
2,Barbour County,Alabama,AlabamaBarbour County
3,Bibb County,Alabama,AlabamaBibb County
4,Blount County,Alabama,AlabamaBlount County


In [63]:
df_c_to_f.head()

,c_state_county,f_state_county_full
0,AlabamaDeKalb County,AlabamaDe Kalb County
1,AlabamaSt. Clair County,AlabamaSt Clair County
2,AlaskaAleutians East Borough,AlaskaAleutians East County
3,AlaskaAleutians West Census Area,AlaskaAleutians West County
4,AlaskaAnchorage Municipality,AlaskaAnchorage County


In [64]:
temp = census_names.merge(final, left_on = 'c_state_county', right_on = 'f_state_county_full', how = 'outer')
census_no_fips = temp[temp['f_state_county_full'].isnull()]

census_no_fips_after_mapping = census_no_fips.merge(df_c_to_f, on = "c_state_county", how= 'outer')
census_no_fips_after_mapping[census_no_fips_after_mapping['f_state_county_full_y'].isnull()]

,CTYNAME,STNAME,c_state_county,fips,county,state_abbr,county_full,state,f_state_county_full_x,f_state_county_full_y
23,Prince of Wales-Hyder Census Area,Alaska,AlaskaPrince of Wales-Hyder Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
final[final['county'].str.contains("awao")]

,fips,county,state_abbr,county_full,state,f_state_county_full
554,15005,Kalawao,HI,Kalawao County,Hawaii,HawaiiKalawao County


In [ ]:
## BROUGHT TO TOP AS HARDCODED!!!!!

additional_mapping = pd.DataFrame(
    np.array([["AlaskaHoonah-Angoon Census Area", "AlaskaHoonah-Angoon County"],
              ["AlaskaKusilvak Census Area", "AlaskaWade Hampton County"],
              ["AlaskaPetersburg Borough", "AlaskaPetersburg Census Area County"],
              ["AlaskaSkagway Municipality", "AlaskaSkagway County"],
              ["AlaskaWrangell City and Borough", "AlaskaWrangell City and County"]]),
    columns=['c_state_county', 'f_state_county_full'])

additional_mapping


### SELECT COLUMNS FOR FINAL

In [66]:
fips = comb[['fips', 'county', 'state_abbr', 'state', 'census_CTYNAME']]

In [67]:
fips = fips[fips['fips'].isnull() == False]

In [68]:
fips = fips[fips['fips'].duplicated() == False]

In [85]:
wales_ind = (fips[fips["county"] == 'Prince of Wales-Hyder']).index.values[0]

#DO I WANT TO DO THIS?!
fips.loc[wales_ind]["census_CTYNAME"] = 'Prince of Wales-Hyder Census Area'

In [69]:
fips.to_csv('../data/county_fips_mapping.csv')

## Researching Missing FIPS inforamtion in final dataset

In [70]:
fips.shape

(3236, 5)

In [89]:
fips[fips['county'].str.contains('Kala')]

#Buena Vista city

,fips,county,state_abbr,state,census_CTYNAME
681,15005,Kalawao,HI,Hawaii,Kalawao County
1325,26077,Kalamazoo,MI,Michigan,Kalamazoo County


In [87]:
fips[(fips['fips'] == '02198') | (fips['fips'] == '02195') ]


,fips,county,state_abbr,state,census_CTYNAME
3172,02195,Petersburg Census Area,AK,Alaska,Petersburg Borough
3173,02198,Prince of Wales-Hyder,AK,Alaska,Prince of Wales-Hyder Census Area


In [ ]:
Prince of Wales-Hyder Census Area

In [ ]:
final[final['fips'] == '66010']

In [ ]:
final['state'].value_counts()

In [96]:
fips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3418 entries, 0 to 3417
Data columns (total 5 columns):
fips              3248 non-null object
county            3248 non-null object
state_abbr        3248 non-null object
state             3248 non-null object
census_CTYNAME    3248 non-null object
dtypes: object(5)
memory usage: 160.2+ KB


In [ ]:
## # of FIPS codes which are not strictly US States
#  District of Columbia          1
#  Guam                          1
#  Virgin Islands                3
#  American Samoa                5
#  Northern Mariana Islands      5
#  Puerto Rico                   76
#
# TOTAL = 3141   #Strict US only

"""
3,007 counties

As of 2016, there were 3,007 counties, 64 parishes, 19 organized boroughs,
10 census areas, 41 independent cities, and the District of Columbia
for a total of 3,142 counties and county-equivalents in the 50 states
and District of Columbia.

"""